In [1]:
import time
import random
import numpy as np
from sklearn.linear_model import LinearRegression
import collections

import altair as alt
import pandas as pd
import plotly.express as px
alt.data_transformers.disable_max_rows()

samplesize = 500
Am, Bm, Cm, = 60, 10, 30
Xm, Ym = 1, 1
Astd,Bstd,Cstd = 11, 15, 21
Xstd,Ystd = 1, 1

m1, k1 = 1, 0
m2, k2 = 1.25, 0
m3, k3 = 0.5, 0
m4, k4 = 1.5, 0

d1 = 1
d2 = 1
d3 = 1
d4 = 1

errorA = 20
errorB = 400
errorC = 25
errorX = 25
errorY = 20

mrange = 0,5  #slope range
erange = 0.2,2 #error range

def jitter(x, scale=1):
    return x + np.random.normal(scale=scale)

def dependent(x, m, c, error=1):
    return jitter(m*x + c, scale=error) # mx +c + error


def linear(n=100):

    def next_generation(A):
        A_ = jitter(A, errorA) # previous A + error
        B_ = dependent(A_, m1, k1, error=errorB)
        C_ = dependent(B_, m2, k2, error=errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[-1]
        A_ = jitter(A, errorA) # previous A + error
        A, _, _ = history[1 - d1]
        B_ = dependent(A, m1, k1, error=errorB)
        _, B, _ = history[1 - d2]
        C_ = dependent(B, m2, k2, error=errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]))
    for i in range(max([d1, d2])):
        history.append(next_generation(A))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def radiating(n=100):
    def next_generation(B):
        B_ = jitter(B, errorB)
        A_ = dependent(B_, m1, k1, errorA)
        C_ = dependent(B_, m2, k2, errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        _, B, _ = history[-1]
        B_ = jitter(B, errorB)
        _, B, _ = history[1-d1]
        A_ = dependent(B, m1, k1, errorA)
        _, B, _ = history[1-d2]
        C_ = dependent(B, m2, k2, errorC)
        return A_, B_, C_


    B = np.random.normal(loc=Bm, scale=Bstd) # normal distribution, Bm - mean and Bstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]))

    for i in range(max([d1, d2])):
        A, B, C = next_generation(B)
        history.append((A, B, C))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def convergent(n=100):

    def next_generation(A, C):
        B_ = jitter(m1*A + m2*C + k1, errorB)
        A_ = jitter(A, errorA)
        C_ = jitter(C, errorC)
        return A_, B_, C_

    def next_generation_delayed(history):
        A, _, _ = history[1-d1]
        _, _, C = history[1-d2]
        B_ = jitter(m1*A + m2*C + k1, errorB)
        A, B, C = history[-1]
        A_ = jitter(A, errorA)
        C_ = jitter(C, errorC)
        return A_, B_, C_

    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    C = np.random.normal(loc=Cm, scale=Cstd) # normal distribution, Cm - mean and Cstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2]))

    for i in range(max([d1, d2])):
        A, B, C = next_generation(A, C)
        history.append((A, B, C))

    for i in range(n):
        history.append(next_generation_delayed(history))

    return np.array(history[-1])


def common_cause(n=100):
    def next_generation(X):
        X = jitter(X, errorX)
        A = dependent(X, m1, k1, errorA)
        B = dependent(X, m2, k2, errorB)
        C = dependent(X, m3, k3, errorC)
        return A, B, C, X

    def next_generation_delayed(history):
        _, _, _, X = history[-1]
        X_ = jitter(X, errorX)
        _, _, _, X = history[1-d1]
        A_ = dependent(X, m1, k1, errorA)
        _, _, _, X = history[1-d2]
        B_ = dependent(X, m2, k2, errorB)
        _, _, _, X = history[1-d3]
        C_ = dependent(X, m3, k3, errorC)
        return A_, B_, C_, X_

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max([d1, d2, d3])):
        A, B, C, X = next_generation(X)
        history.append((A, B, C, X))

    for i in range(n):
        history.append(next_generation_delayed(history))

    A, B, C, _ = history[-1]
    return np.array([A,B,C])


def single_difference_cause(n=100):

    def next_generation(A, X):
        X = jitter(X, errorX)
        A = jitter(A, errorA)
        B = jitter(m1*A + m2*X + k1, errorB)
        C = dependent(X, m3, k3, errorC)
        return A, B, C, X

    def next_generation_delayed(history):
        A, _, _, X = history[-1]
        A_ = jitter(A, errorA)
        X_ = jitter(X, errorX)

        A, _, _, _ = history[1-d1]
        _, _, _, X = history[1-d2]
        B_ = jitter(m1*A + m2*X + k1, errorB)

        _, _, _, X = history[1-d3]
        C_ = dependent(X, m2, k3, errorC)

        return A_, B_ , C_, X_

    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    A = np.random.normal(loc=Am, scale=Astd) # normal distribution, Am - mean and Astd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3]))
    for i in range(max(d1, d2, d3)):
        A,B,C,X = next_generation(A,X)
        history.append((A, B, C, X))

    for i in range(n):
        history.append(next_generation_delayed(history))

    A, B, C, X = history[-1]
    return np.array([A, B, C])


def double_difference_cause(n=100):
    def next_generation(X,Y):
        X = jitter(X, errorX)
        Y = jitter(Y, errorY)
        A = dependent(X, m1, k1, errorA)
        B = jitter(m2*X+m3*Y+k2, errorB)
        C = dependent(Y, m4, k4, errorC)
        return A, B, C, X, Y

    def next_generation_delayed(history):
        _, _, _, X, Y = history[-1]
        X_ = jitter(X, errorX)
        Y_ = jitter(Y, errorY)

        _, _, _, X, _ = history[1-d1]
        A_ = dependent(X, m1, k1, errorA)

        _, _, _, X, _ = history[1-d2]
        _, _, _, _, Y = history[1-d3]
        B_ = jitter(m2*X+m3*Y+k2, errorB)

        _, _, _, _, Y = history[1-d4]
        C_ = dependent(Y, m4, k4, errorC)
        return A_, B_, C_, X_, Y_


    X = np.random.normal(loc=Xm, scale=Xstd) # normal distribution, Xm - mean and Xstd - standard deviation
    Y = np.random.normal(loc=Ym, scale=Ystd) # normal distribution, Ym - mean and Ystd - standard deviation
    history = collections.deque(maxlen=max([d1, d2, d3, d4]))

    for i in range(max([d1, d2, d3, d4])):
        A, B, C, X, Y = next_generation(X, Y)
        history.append((A, B, C, X, Y))

    for i in range(n):
        history.append(next_generation_delayed(history))

    A, B, C, X, Y = history[-1]
    return np.array([A, B, C])


def simulations_data(pathway, n=1000):
    random.seed(time.time())
    return np.array([pathway() for i in range(n)])


def regress(X,Y):
    model = LinearRegression()
    mXY = model.fit(X.reshape(-1,1), Y)
    r_sqr = mXY.score(X.reshape(-1,1), Y)
    residual = Y - model.predict(X.reshape(-1, 1))
    return mXY.intercept_, mXY.coef_[0], r_sqr, residual


def get_slope_intercept(model):
    return model._slopt, model._intercept

def compute_regresssion(ABC):
    A,B,C = ABC.transpose()
    RAB = regress(A, B)
    RBC = regress(B, C)
    RAC = regress(A, C)
    corrE = np.corrcoef(np.array([RAB[3], RBC[3]]))
    corrE_BA_C = np.corrcoef(np.array([np.square(RAB[3]), C]))


    #print(RAB[1]*RBC[1]-RAC[1]) ## better to look at distribution of this error..it should come with center as 0
    return {"kAB": RAB[0], "kBC":RBC[0], "kAC":RAC[0],
            "mAB":RAB[1], "mBC":RBC[1], "mAC":RAC[1],
            "r_sqrAB":RAB[2], "r_sqrBC":RBC[2], "r_sqrAC":RAC[2],
            "r_E":corrE[0,1],
            "r_E_BA_C":corrE_BA_C[0,1],
            "n":len(A)}

def compute_correlation(ABC):
    corr = np.corrcoef(ABC.transpose())
    rAB, rBC, rAC = corr[0,1], corr[1,2], corr[0,2]
    #print(rAB**2*rBC**2-rAC**2) ## better to look at distribution of this error..it should come with center as 0
                                ## or see correlation between these two quantities should be 1 and if we regress ,
                                ###       it should have slope 1
    return {"rAB":rAB, "rBC":rBC, "rAC":rAC}

def update_slopes_errors():
    def select(lower, upper):
        return lower + random.random()*(upper-lower)
    
    global m1, m2, m3, m4
    global errorA, errorB, errorC, errorX, errorY

    m1 = select(*mrange)
    m2 = select(*mrange)
    m3 = select(*mrange)
    m4 = select(*mrange)
    
    errorA = select(*erange)
    errorB = select(*erange)
    errorC = select(*erange)
    errorX = select(*erange)
    errorY = select(*erange)

def overall_simulation(n=100):
    stats = []
    data = []
    for i in range(n):
        update_slopes_errors()
        ABC = simulations_data(convergent, samplesize) # you can change pathway here
        r = compute_regresssion(ABC)
        r.update(compute_correlation(ABC))
        stats.append(r)
        data.append(ABC)
    return pd.DataFrame(stats), np.array(data)

d, ABC_all = overall_simulation()

In [2]:
def compute_confidence_interval(r, n):
    def boundary(zeta):
        return ((np.exp(2*zeta))-1)/((np.exp(2*zeta))+1)

    z = 0.5*np.log(((1-r)/(1+r)))
    zetal = z-1.96*np.sqrt(1/(n-3))
    rl = boundary(zetal)
    zetau = z+1.96*np.sqrt(1/(n-3))
    ru = boundary(zetau)
    return rl, ru

def confidence_status(L, U, v):
    l = v.where(v < L, "less")
    l.where(v > U, "more", inplace=True)
    w = l.where((v>= L) & (v <= U), "within")
    return w

def confidence_status_(r, n):
    L, U = compute_confidence_interval(r, n)
    return confidence_status(L, U, r)


def add_confidence_stats(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    L,U = compute_confidence_interval(d.rBC**2, d['n'])
    d['confidence_residual_corr'] = confidence_status(L, U, d.r_E)
    L,U = compute_confidence_interval(d.r_E, d['n'])#???
    d['confidence_corrected_bc_corr'] = confidence_status(L, U, d.r_E_BA_C**2)

def confidence_graphs(d):
    confidence = alt.Chart(d, title=f"Correlation confidence{(d['confidence']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_res_corr = alt.Chart(d, title=f"residual corr {(d['confidence_residual_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_residual_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    confidence_corrected_bc_corr = alt.Chart(d, title=f"corrected bc corr {(d['confidence_corrected_bc_corr']=='within').sum()}/{len(d)}").mark_point().encode(
        x = alt.X('rAB2:Q'),
        y = alt.Y('rBC2:Q'),
        color = alt.Color('confidence_corrected_bc_corr:N',
                          scale=alt.Scale(domain=['less', 'within', 'more'],
                                          range=['orange', 'green','red'])),
    ).transform_calculate(
        rAB2='datum.rAB*datum.rAB',
        rBC2='datum.rBC*datum.rBC'
    )
    
    return alt.vconcat(confidence,confidence_res_corr, confidence_corrected_bc_corr)

add_confidence_stats(d)
confidence_graphs(d)

alt.VConcatChart(...)

In [3]:
def stats_graphs(d):
    d['rAB2*rBC2-rAC2']=d.rAB**2 * d.rBC**2 - d.rAC**2
    d['r_E_BA_C2-rBC2'] = d.r_E_BA_C**2 - d.rBC**2
    #d['rAC2'] = d.rAC**2
    d['mAB*mBC-mAC'] = d.mAB*d.mBC - d.mAC
    L,U = compute_confidence_interval(d.rAC**2, d['n'])
    d['confidence'] = confidence_status(L, U, d.rAB**2*d.rBC**2)
    slope_histogram = alt.Chart(d).mark_bar().encode(
        x=alt.X('mAB*mBC-mAC:Q', bin=True),
        y='count()').properties(title="slope diff histogram")

    bincount = 100
    ticks = 10
    correlation_graph = alt.Chart(d).mark_bar().encode(
        x=alt.X('rAB2*rBC2-rAC2:Q',bin=True,
                   axis=alt.Axis(
                    tickCount=ticks,
                    grid=False)),
        y='count()').properties(
            title="Correlation")
    residual_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X('r_E:Q', bin=True),
        y='count()').properties(title="Correlation of residuals")
    corrected_correlation = alt.Chart(d).mark_bar().encode(
        x=alt.X("r_E_BA_C2-rBC2:Q", bin=True),
        y='count()').properties(
        title="Corrected Correlation")

    return alt.vconcat(slope_histogram,correlation_graph,residual_correlation, corrected_correlation)

stats_graphs(d)

alt.VConcatChart(...)

In [4]:
A, B, C = random.choice(ABC_all).transpose()
AB = alt.Chart(pd.DataFrame({"A":A, "B":B})).mark_circle().encode(
    x="A",
    y="B")
BC= alt.Chart(pd.DataFrame({"B":B, "C":C})).mark_circle().encode(
    x="B",
    y="C")
AC = alt.Chart(pd.DataFrame({"A":A, "C":C})).mark_circle().encode(
    x="A",
    y="C")

alt.vconcat(AB, BC, AC)

alt.VConcatChart(...)

In [5]:
d

,kAB,kBC,kAC,mAB,mBC,mAC,r_sqrAB,r_sqrBC,r_sqrAC,r_E,...,n,rAB,rBC,rAC,rAB2*rBC2-rAC2,r_E_BA_C2-rBC2,mAB*mBC-mAC,confidence,confidence_residual_corr,confidence_corrected_bc_corr
0,119.354563,48.439685,28.422809,-3.788991,0.186098,-0.000955,0.211467,0.785649,3.046774e-07,0.455297,...,500,-0.459855,0.886368,-0.000552,0.166138,-0.780945,-0.704168,within,within,within
1,-115.407024,-34.474020,31.389389,-3.824279,-0.189765,-0.006064,0.301318,0.693687,1.459479e-05,-0.547013,...,500,-0.548924,-0.832879,-0.003820,0.209006,-0.692585,0.731778,within,within,within
2,-157.417993,28.369632,33.444828,2.525177,-0.187924,-0.072816,0.126413,0.902967,2.687627e-03,-0.349278,...,500,0.355547,-0.950246,-0.051842,0.111460,-0.902876,-0.401726,within,within,within
3,-57.473269,57.079745,25.433520,3.479478,-0.175853,0.093151,0.559949,0.366646,4.758204e-03,-0.740014,...,500,0.748298,-0.605513,0.068980,0.200545,-0.364512,-0.705029,within,within,within
4,63.836587,5.348641,31.534844,-0.244255,0.493186,-0.033059,0.004758,0.996876,3.572540e-04,0.061823,...,500,-0.068980,0.998437,-0.018901,0.004386,-0.996867,-0.087404,more,within,more
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,76.463880,29.575033,25.276974,-1.230428,0.304948,0.083247,0.067195,0.901882,2.983012e-03,0.257729,...,500,-0.259219,0.949675,0.054617,0.057619,-0.901716,-0.458464,more,within,within
96,-74.702690,-28.779119,28.054373,-3.445345,-0.201598,0.015121,0.476151,0.532097,1.200712e-04,-0.686279,...,500,-0.690037,-0.729450,0.010958,0.253239,-0.531155,0.679454,within,within,within
97,42.872460,72.689424,28.959890,-4.687060,0.182223,0.007777,0.729515,0.264073,1.597159e-05,0.847898,...,500,-0.854116,0.513880,0.003996,0.192629,-0.255707,-0.861866,within,within,within
98,-72.977222,-31.152516,33.281138,-3.745426,-0.208090,-0.030988,0.532770,0.444512,3.743761e-04,-0.724956,...,500,-0.729911,-0.666717,-0.019349,0.236448,-0.433687,0.810373,within,within,within
